In [1]:
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1639110915514_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/d7/24/4b70d13b553f7ca14efaa652f5b61e8a1fc63d5a98a0b4eed74eca5c9a16/boto3-1.20.23-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ab/84/fc3717a7b7f0f6bb08af593127171f08e3e0087c197922da09c01bfe7c3a/s3transfer-0.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/06/75/72b9236e2ff5ff7d24d1a04dae90cc5974f1cb830ddfe61ba8a18df4664d/botocore-1.23.23-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/af/f4/524415c0744552cce7d8bf3669af78e8a069514405ea4fcbd0cc44733744/urllib3-1.26.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

You are using pip version 9.0.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [2]:
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
s3 = boto3.resource('s3')
bucket = 'lsc-sentiments-final-project'
bucket_resource = s3.Bucket(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
data = spark.read.json('s3://lsc-sentiments-final-project')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 12
Total Rows: 26
root
 |-- anger: string (nullable = true)
 |-- date: string (nullable = true)
 |-- disgust: string (nullable = true)
 |-- fear: string (nullable = true)
 |-- id: string (nullable = true)
 |-- joy: string (nullable = true)
 |-- love: long (nullable = true)
 |-- others: string (nullable = true)
 |-- sadness: string (nullable = true)
 |-- sentiment: struct (nullable = true)
 |    |-- results: string (nullable = true)
 |-- surprise: string (nullable = true)
 |-- tweet: string (nullable = true)

In [25]:
data[['date', 'anger', 'disgust', 'fear', 'joy', 'others', 'sadness', 'surprise']].show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+-------+------+------+------+-------+--------+
|               date| anger|disgust|  fear|   joy|others|sadness|surprise|
+-------------------+------+-------+------+------+------+-------+--------+
|2021-12-09 22:57:00|0.6234| 0.1185|0.0236|0.0066|0.0964| 0.1169|  0.0147|
|2021-12-09 22:57:01|0.0279| 0.0061|0.0039|0.5122|0.3216| 0.0961|  0.0322|
|2021-12-09 22:56:49|0.2702| 0.0484|0.0139|0.0306|0.5822| 0.0263|  0.0283|
|2021-12-09 22:56:59|0.2994| 0.0784|0.0356|0.0076|0.2465| 0.3131|  0.0192|
|2021-12-09 22:56:49|0.2132| 0.0279|0.0062|0.0514|0.3416| 0.3492|  0.0105|
|2021-12-09 22:56:51|0.0429|  0.013|0.0103|0.0102|0.8808| 0.0264|  0.0164|
|2021-12-09 22:56:59|0.1378| 0.0581|  0.12|0.0663|0.3155| 0.0729|  0.2293|
|2021-12-09 22:56:57|0.4075| 0.0878|0.0245|0.0092|0.3334| 0.1222|  0.0154|
|2021-12-09 22:56:48|0.0368| 0.0102| 0.017|0.0117|0.8629| 0.0216|  0.0398|
|2021-12-09 22:56:46|0.5093| 0.0609|0.0131| 0.032|0.3379| 0.0118|  0.0351|
|2021-12-09 22:56:47|0.62

In [35]:
from pyspark.sql.functions import split
data = data.withColumn('date_day', split(data.date, " ")[0])
data = data.withColumn('anger_n', data.anger.cast("float"))
data = data.withColumn('disgust_n', (data.disgust).cast("float"))
data = data.withColumn('fear_n', (data.fear).cast("float"))
data = data.withColumn('joy_n', (data.joy).cast("float"))
data = data.withColumn('others_n', (data.others).cast("float"))
data = data.withColumn('sadness_n', (data.sadness).cast("float"))
data = data.withColumn('surprise_n', (data.surprise).cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
data_to_graph = data[['date_day', 'anger_n', 'disgust_n', 'fear_n', 'joy_n', 'others_n', 'sadness_n', 'surprise_n']].groupBy('date_day').avg('anger_n', 'disgust_n', 'fear_n', 'joy_n', 'others_n', 'sadness_n', 'surprise_n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…